# 无效的工具结果

您传递给模型的 [`ToolMessages`](https://api.js.langchain.com/classes/_langchain_core.messages_tool.ToolMessage.html) 数量过多、过少或不匹配。

当[使用模型调用工具](/docs/concepts/tool_calling)时，模型响应返回的 [`AIMessage`](https://api.js.langchain.com/classes/_langchain_core.messages.AIMessage.html)
将包含一个 `tool_calls` 数组。为了继续流程，您下次传递给模型的消息必须为该数组中的每个项目精确地包含一个 `ToolMessage`，其中包含该工具调用的结果。每个 `ToolMessage` 必须有一个 `tool_call_id` 字段，
其值需与 `AIMessage` 中的某个 `tool_calls` 匹配。

例如，假设模型返回以下响应：

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { ChatOpenAI } from "@langchain/openai";
import { BaseMessageLike } from "@langchain/core/messages";

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
});

const dummyTool = tool(
  async () => {
    return "action complete!";
  },
  {
    name: "foo",
    schema: z.object({}),
  }
);

const modelWithTools = model.bindTools([dummyTool]);

const chatHistory: BaseMessageLike[] = [
  {
    role: "user",
    content: `Call tool "foo" twice with no arguments`,
  },
];

const responseMessage = await modelWithTools.invoke(chatHistory);

console.log(responseMessage);

AIMessage {
  "id": "chatcmpl-AIgT1xUd6lkWAutThiiBsqjq7Ykj1",
  "content": "",
  "additional_kwargs": {
    "tool_calls": [
      {
        "id": "call_BknYpnY7xiARM17TPYqL7luj",
        "type": "function",
        "function": "[Object]"
      },
      {
        "id": "call_EHf8MIcTdsLCZcFVlcH4hxJw",
        "type": "function",
        "function": "[Object]"
      }
    ]
  },
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 42,
      "completionTokens": 37,
      "totalTokens": 79
    },
    "finish_reason": "tool_calls",
    "usage": {
      "prompt_tokens": 42,
      "completion_tokens": 37,
      "total_tokens": 79,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0
      }
    },
    "system_fingerprint": "fp_e2bde53e6e"
  },
  "tool_calls": [
    {
      "name": "foo",
      "args": {},
      "type": "tool_call",
      "id": "call_BknYpnY7xiARM17TPYqL7luj"
    },
    {
      "na

仅使用一个工具响应调用模型会导致错误：

In [2]:
const toolResponse1 = await dummyTool.invoke(responseMessage.tool_calls![0]);

chatHistory.push(responseMessage);
chatHistory.push(toolResponse1);

await modelWithTools.invoke(chatHistory);

BadRequestError: 400 An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_EHf8MIcTdsLCZcFVlcH4hxJw
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/error.js:45:20)
    at OpenAI.makeStatusError (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:291:33)
    at OpenAI.makeRequest (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:335:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /Users/jacoblee/langchain/langchainjs/libs/langchain-openai/dist/chat_models.cjs:1441:29
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'access-control-expose-headers': 'X-Request-ID',
    'alt-svc': 'h3=":443"; ma=8640

如果我们添加第二个响应，调用将会按预期成功，因为现在每个工具调用都有一个对应的工具响应：

In [3]:
const toolResponse2 = await dummyTool.invoke(responseMessage.tool_calls![1]);

chatHistory.push(toolResponse2);

await modelWithTools.invoke(chatHistory);

AIMessage {
  "id": "chatcmpl-AIgTPDBm1epnnLHx0tPFTgpsf8Ay6",
  "content": "The tool \"foo\" was called twice, and both times returned the result: \"action complete!\".",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 98,
      "completionTokens": 21,
      "totalTokens": 119
    },
    "finish_reason": "stop",
    "usage": {
      "prompt_tokens": 98,
      "completion_tokens": 21,
      "total_tokens": 119,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0
      }
    },
    "system_fingerprint": "fp_e2bde53e6e"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 21,
    "input_tokens": 98,
    "total_tokens": 119,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_details": {
      "reasoning": 0
    }
  }
}


但如果添加了一个重复的额外工具响应，调用会再次失败：

In [4]:
const duplicateToolResponse2 = await dummyTool.invoke(responseMessage.tool_calls![1]);

chatHistory.push(duplicateToolResponse2);

await modelWithTools.invoke(chatHistory);

BadRequestError: 400 Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/error.js:45:20)
    at OpenAI.makeStatusError (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:291:33)
    at OpenAI.makeRequest (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:335:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /Users/jacoblee/langchain/langchainjs/libs/langchain-openai/dist/chat_models.cjs:1441:29
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'access-control-expose-headers': 'X-Request-ID',
    'alt-svc': 'h3=":443"; ma=86400',
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31d57dff5e0f3b-EWR',
    connection:

如果 `ToolMessages` 前面没有 `AIMessage` 调用工具，则不应将其再次传递给模型。例如，以下代码将失败：

In [5]:
await modelWithTools.invoke([{
  role: "tool",
  content: "action completed!",
  tool_call_id: "dummy",
}])

BadRequestError: 400 Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/error.js:45:20)
    at OpenAI.makeStatusError (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:291:33)
    at OpenAI.makeRequest (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:335:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /Users/jacoblee/langchain/langchainjs/libs/langchain-openai/dist/chat_models.cjs:1441:29
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'access-control-expose-headers': 'X-Request-ID',
    'alt-svc': 'h3=":443"; ma=86400',
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31d5da7fba19aa-EWR',
    connection:

有关工具调用的更多详细信息，请参阅[本指南](/docs/how_to/tool_results_pass_to_model/)。

## 故障排除

以下方法可能有助于解决此错误：

- 如果你使用的是自定义执行器，而不是像LangGraph的[`ToolNode`](https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)
  或传统LangChain [AgentExecutor](/docs/how_to/agent_executor)这样的预构建执行器，请确认你正在为每次工具调用调用并返回一个工具的结果。
- 如果你正在手动创建消息并使用带有[少量工具调用示例](/docs/how_to/tools_few_shot)，并且想要模拟失败情况，
  你仍需返回一个内容表明失败的`ToolMessage`。
